Detecting Sarcasm with RoBERTa Embeddings and XGBoost
---------------------------------------------------------------------------

By Ivan T. Ivanov, Lisamaria Cutrone & Jennifer Passarelli

A Program Comprehensive Assessment

Submitted to Ivan T. Ivanov, Mathematics Department, Vanier College, December 2024:

In [ ]:
!pip install -qU flair umap-learn matplotlib

Abstract
-----------------------------------

This project aims to develop a model capable of distinguishing between sarcastic and non-sarcastic text in article headlines. A Kaggle dataset of sarcastic headlines from The Onion and non-sarcastic headlines from HuffPost was used. RoBERTa, a transformer-based language model, was used to embed the headlines to capture semantic relationships between the headlines and converting them into numerical vectors. These embeddings were then used to train an XGBoost classifier.

The results demonstrate that the model is effective in detecting sarcasm with high precision, recall, and F1-scores above 0.8. This indicates the model's accurate identification of sarcastic headlines while minimizing false positives and negatives. This model can have applications in social media content moderation and sentiment analysis, aiding in tasks like detecting hate speech or flagging inappropriate content and predicting the emotional tone of text.

While the model exhibits high accuracy, potential improvements include experimenting with other embedding models such as BERT, XLNet, or ELMo, and expanding the dataset for broader sarcastic expression detection.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.metrics import classification_report
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import flair
from flair.data import Sentence
from umap import UMAP
import torch

The objective is to obtain a model able to distinguish and classify sarcastic and non-sarcastic text. Specifically, in article headlines.  

To do this, a dataset containing sarcastic headlines from The Onion and non-sarcastic headlines from HuffPost was first embedded and then trained in XGBoost.

In [ ]:
df = pd.read_json('Sarcasm.json', lines=True)

In [ ]:
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [ ]:
type(df['is_sarcastic'][0])

numpy.int64

In [ ]:
df.drop(['article_link'], axis=1, inplace=True)
df.describe()

,is_sarcastic
count,28619.000000
mean,0.476397
std,0.499451
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


### The headlines in sample_df will be used to plot the text embeddings in 3d

In [ ]:
_, sample_df, _, sample_labels = train_test_split(df.loc[:, df.columns != "is_sarcastic"],
                                                  df["is_sarcastic"],
                                                  test_size=0.01,
                                                  stratify=df["is_sarcastic"]) #Ensures that the distribution of sarcastic and non-sarcastic samples is the same in the sample as in the original dataset

In [ ]:
sample_df

,headline
24870,gop mulls forcing christine blasey ford to pub...
5010,noel comrie's gps guide on positive self-affir...
3345,area man incapable of making plans without exc...
737,85 percent of u.s. cole slaw remains uneaten
13153,new diet surge targets overweight snowboarders
...,...
17223,"gruff, no-nonsense teacher only hard on studen..."
1536,"obama blasted by cool, refreshing air"
13665,dermatologists recommend regularly checking bo...
19525,iconic '90s band helped this couple open up ab...


In [ ]:
sample_labels

,is_sarcastic
24870,1
5010,0
3345,1
737,1
13153,1
...,...
17223,1
1536,1
13665,1
19525,0


RoBERTa and Text Embeddings
-----------------------------------------------------

Since machine learning algorithms have a mathematical foundation, they do not operate well with words. To allow the model training to be done on the data, the data must be first converted into something XGBoost can process: numerical vectors.  

It is possible to assign a random number to every word present in the dataset, however, this might result in words that are similar in meaning or synonymous having very different numbers.

To simplify the complexity of XGBoosts’ forest, and to help it process words correctly, words that are similar in meaning should have similar numbers.

Additionally, it is advantageous for words to be assigned more than one number, depending on what context they are used for. For example, the word Great can have two numbers assigned to it: it can be used in a positive way, or in a sarcastic way depicting a negative situation.  

Doing this is impossibly long to do manually. Therefore, the program imports the TransformerDocumentEmbeddings class from the flair.embeddings module to capture semantic relationships between words in the dataset and specifies the 'roberta-base' as the underlying transformer model to generate text embeddings (word to vector conversion).  

These text embeddings will represent the words in a dense vector to assure that similar words stay together and can then be used as input for XGBoost to classify headlines as sarcastic or not.

In [ ]:
from flair.embeddings import TransformerDocumentEmbeddings

embedding = TransformerDocumentEmbeddings('roberta-base')

# create a sentence
sentence = Sentence('The grass is green or possible not.')

pupa = embedding.embed(sentence)

In [ ]:
#pupa[0].embedding

RoBERTa is a natural language processing model designed to analyze and understand text. This model encodes its meaning into 768-dimensional vectors. The first step in using this model is called tokenization. The tokenizer takes either words or subwords and splits them into tokens. This process converts raw text into smaller units that the model can process. This process is crucial for breaking down the text into manageable and meaningful pieces. Each token is converted into a 768-dimensional vector which is an embedding that encodes its identity and position (the context of nearby tokens).

To continue, the tokenized input is then passed into the encoder, which is a neural network containing a series of neuron layers. The encoder uses neighbourhood awareness, a self-attention mechanism. Each token’s vector is updated by considering all the other tokens in the input. This process helps the model capture semantic relationships between tokens, even across long sequences, within a context window of up to 512 tokens.

The encoder computes a weighted combination of these surrounding token vectors, assigning more importance to tokens that are most relevant to the current token. During training, the neural network learns these weights by adjusting how much attention each token pays to others in the sequence. These learned weights enable the model to capture the relationships and context between tokens to make accurate predictions. Each token's updated vector is a linear combination of all the token vectors in the sequence. This means the 768 components of the vector are computed as combinations of the corresponding components of nearby tokens. This helps capture complex relationships, such as a word's meaning in a sentence's context.

After computing the weighted combinations, the encoder applies non-linear functions through neural network layers. These transformations add flexibility and allow the model to learn more complex patterns. They allow the model to go beyond simple linear relationships and capture intricate dependencies between words and sentences.

RoBERTa’s encoder consists of 12 layers. In each layer, the token vectors are refined further. Each layer does two main things: mixing the vectors and applying non-linear transformations. Through each layer, this "mixing" makes the vector for each token increasingly context-aware, blending information from its surrounding tokens. After 12 layers, the model ideally has produced semantic vectors. Each of these vectors no longer represents just the original token. They now reflect the full context of the token, capturing both its position in the sequence and its relationship to every other token in the input. At this point, each token's vector is semantically aware, meaning it understands its meaning in the context of the entire sentence or passage.


In [ ]:
len(pupa[0].embedding)

768

In [ ]:
def get_embeddings(texts):
    all_embeddings = []
    print(f"Total number of records: {len(texts)}")

    # Extract embeddings for the texts in batches
    for element in texts:

        sentence = Sentence(element)

        embedi = embedding.embed(sentence)
        array = embedi[0].embedding

        all_embeddings.append(array)

    return all_embeddings

In [ ]:
embeddings = get_embeddings(sample_df['headline'])

Total number of records: 287


In [ ]:
type(embeddings), len(embeddings), embeddings[0].shape

(list, 287, torch.Size([768]))

In [ ]:
embeddings_array = np.array([x.cpu() for x in embeddings])

In [ ]:
embeddings_array.shape

(287, 768)

UMAP and 3D Scatter Plot
-------------------

To get a visual representation of the relationship between the headlines, the RoBERTa embeddings can be plotted to see how headlines cluster based on their similarities in meaning. However, 768 dimensions are difficult to visualise. Therefore, the program uses UMAP, or Uniform Manifold Approximation and Projection to reduce the dimensionality of the embeddings from 768 to 3.


## <font color="red">EXPLAIN BRIEFLY HOW UMAP WORKS:

In [ ]:
reducer = UMAP(n_neighbors=5, n_components=3, metric='cosine', min_dist=0.05, spread=0.5)
reduced_embeddings = reducer.fit_transform(embeddings_array)

In [ ]:
x, y, z = reduced_embeddings[:,0], reduced_embeddings[:,1], reduced_embeddings[:,2]

In [ ]:
labels = ['not sarcastic', 'sarcastic']
unique_labels = np.unique(sample_labels)

In [ ]:
colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_labels)))
label_color_dict = dict(zip(unique_labels, colors))

In the 3D scatter plot, each point represents a headline where the red are sarcastic and the purple are non-sarcastic. The point's position in the space is
determined by the reduced embedding obtained from the UMAP. Although the embedding is reduced, the plot preserves the relationship between the points as much as possible. This means that headlines (points) that are similar in meaning (share close sematic relationships) will be located near each other in the (x,y,z) plot.

The purpose of the plot is to visually determine if there is a distinction between the sarcastic and non-sarcastic headlines.

In a perfect model, all of the points representing the sarcastic headlines would be clustered together and all of the points representing the the non-sarcastic headlines would be clustered distinctively from the others (clear seperation).

In [ ]:
import plotly.graph_objects as go

# Assuming you have x, y, z, and sample_labels defined

# Create a Plotly figure
fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        color=[label_color_dict[label] for label in sample_labels],  # Color by label
        size=5  # Adjust marker size as needed
    )
)])

# Set axis labels and title
fig.update_layout(scene=dict(
    xaxis_title='X',
    yaxis_title='Y',
    zaxis_title='Z'
),
    title='3D Scatter Plot of Text Embeddings')

# Show the interactive plot
fig.show()

The 3D projection does not show a clear class separation between the headlines. This suggests that the model might not effectively distiguish between sarcastic and non-sarcastic headlines. However, this conclusion may be misleading because it is important to acknowledge that some information might have been lost in the process of reducing the embedding's dimension from $R^{768}$ to $R^{3}$.
Therefore, the separation might be more evident in its' original dimensional space or in dimensions higher than 3.

Training and Testing Subsets
----------------------------------

## <font color="red">Now lets split the dataset in train and test subsets (why is this necessary?)</font>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != "is_sarcastic"], df["is_sarcastic"],
                                                    test_size=0.3,
                                                    stratify=df["is_sarcastic"])

In [ ]:
len(X_train), len(X_test)

(20033, 8586)

In [ ]:
# Get embeddings for the training and test set
train_embeddings = get_embeddings(texts=X_train["headline"].tolist())
#train_embeddings_df = pd.DataFrame(train_embeddings)
train_embeddings_array = np.array([x.cpu() for x in train_embeddings])

test_embeddings = get_embeddings(texts=X_test["headline"].tolist())
#test_embeddings_df = pd.DataFrame(test_embeddings)
test_embeddings_array = np.array([x.cpu() for x in test_embeddings])

Total number of records: 20033
Total number of records: 8586


## <font color="red">We have the 768-dimensional embeddings of the train and test headlines. Time to train a classifier.</font>

## <font color="black">The XGBoost algorithm</font>

XGBoost, or eXtreme Gradient Boosting, is a powerful machine learning algorithm that excels in both classification and regression. It is widely used in competitive machine learning due to its speed and predictive performance. For tasks such as detecting sarcasm in headlines, XGBoost's XGBClassifier can be employed with the objective parameter set to binary:logistic for binary classification since the model decides whether the headline is sarcastic or not.

More specifically, in this project, sarcasm detection is achieved by leveraging XGBoost with features extracted from text data using RoBERTa. The sarcasm detection prediction is made by first imputing <font color="red">! imputing means something else</font> the 768-dimensional vectors obtained from RoBERTa. The model starts with 768 features derived from the text. These vectors are put into the trained XGBoost model. Each decision tree processes the vector and outputs a score. This indicates the likelihood that the headline is sarcastic. It does so in such a way that it assigns a higher score to headlines that it believes are sarcastic and lower scores to those that are not. All the individual tree outputs are summed to produce a final score. It combines the values from the leaves to assign a score to each example, which represents the probability that the headline is sarcastic.

To measure the effectiveness of the model and guide the training process, XGBoost uses an objective function, which consists of two parts: training loss and regularization. The training loss measures how well the model fits the data, while the regularization term penalizes overly complex models to prevent overfitting. In classification tasks, the training loss (L) is often defined using logistic loss, which quantifies the error between the predicted probability and the actual label:

$$ \mathcal{L} = \sum_{i=1}^{N} \left[ -y_i \log(\hat{y}_i) - (1 - y_i) \log(1 - \hat{y}_i) \right] $$

where $$-y_i$$ is the actual value for the i-th training instance

and  $$\hat{y}_i$$is the predicted probability for the i-th instance

This is how XGBoost works at the final level (sum).
However, at every leaf, the elements to be summed are obtained using similarity scores.

Like Gradient Boosting and AdaBoost, XGBoost builds a series of decision trees based on the previous tree. Each tree is said to “correct a bit” an initial prediction to obtain an improved one. The main idea is to make a prediction a little bit better at each tree.

The first step to training a model with XGBoost is to make an initial prediction. Unlike other machine learning algorithms where the initial prediction is the average of the variable factor, XGBoost uses 0.5 as a standard first prediction. For binary classification, an initial prediction of 0.5 reflects the assumption that, without prior knowledge, the model considers each class (sarcastic or not) equally likely. As training progresses, this initial guess is refined based on the residuals of previous trees. However, the initial prediction can be changed by the programmer.

Then, the residuals, which are the differences between the observed and the predicted value, show the initial predictions’ accuracy. Higher residuals show a greater difference between what was predicted and what was observed, meaning the observed result was further from what was predicted.

The Similarity Score in XGBoost is directly related to the objective function. This score can be viewed as a measure of how well the model has fit the data at that stage of the learning process. The formula for the similarity score involves a regularization term λ and a probability value

P, which is the predicted value from the previous tree. The score is defined as:

$$ \text{Similarity score} = \frac{(\sum \text{Residuals})^2}{\sum^N [p(1-p)]+\lambda} $$
​

Here, the residuals are used to evaluate how much error is present, and the regularization term λ ensures that the tree does not become overly complex. The Similarity Score helps determine how well each split in the decision tree minimizes the residual error. The objective function, which consists of both the training loss and regularization, drives the optimization of this score.

Now, the tree continues to build by predicting the variable factor at different thresholds. This means XGBoost splits the data by clustering similar residuals together and separates the residuals into two groups. The residuals on the right side of the threshold are used to build another residual leaf, and the residuals on the left do the same.

Again, a similarity score is calculated for each leaf.

To compare the changes in similarity score, what is known as a Gain can be calculated.

$$\text{Gain} = \text{Previous Score} - (\text{Left Score} + \text{Right Score})$$

In decision trees, gain refers to the improvement in the model’s performance or the error reduction when a dataset is split at a specific node. The gain helps in selecting which feature to split on and where to split for each decision node. It identifies the optimal way to split decision tree leaves by calculating the best gain from each potential split. A higher gain indicates a more valuable split.

This process repeats until further splitting is no longer beneficial.




Once the tree has been built, we need to convert the model's log(odds) prediction to a probability.

To convert a log(odds) value into a probability, the following logistic function is used:

$$\hat{y} = \frac{1}{1 + e^{-z}}$$
 Where
𝑧 is the log(odds) score predicted by the model

$$z = \log \left( \frac{p}{1 - p} \right)$$

The sigmoid (logistic) function maps any input into a probability value between 0 and 1. A value closer to 1 indicates a higher likelihood of being sarcastic. On the other hand, a value closer to 0 indicates a higher likelihood of being non-sarcastic. Once the sigmoid function provides a probability, it is compared to a threshold which is commonly 0.5. If the probability is greater than 0.5, the headline is classified as sarcastic. If the probability is less than or equal to 0.5, the headline is classified as non-sarcastic.

The probability obtained from the logistic function should result in a smaller residual than the previous prediction.

This process is repeated until the maximum number of trees is reached. The size of the forest can be set using the parameter num_boost_round. <font color="red">Explain the hyperparameter grid search.</font>

In [ ]:
def train_model(data: pd.DataFrame, labels: pd.Series):
    if torch.cuda.is_available():
        boost_device = "cuda"
    else:
        boost_device = "cpu"

    # Initialize the XGBoost Classifier
    xgb_clf = xgb.XGBClassifier(objective="binary:logistic",
                                device=boost_device,
                                random_state=3137)

    # Define hyperparameters and values to tune
    param_grid = {
        'max_depth': [5, 6, 7, 8],
        'eta': np.arange(0.05, 0.3, 0.05)
    }

    print(f"Number of rows in training data: {len(data)}")

    # Perform hyperparameter tuning using GridSearchCV
    grid_search = GridSearchCV(estimator=xgb_clf, param_grid=param_grid, scoring="roc_auc",
                               cv=5, verbose=3)
    grid_search.fit(data, labels)

    # Get the best hyperparameters
    best_max_depth = grid_search.best_params_['max_depth']
    print(f"Best max depth: {best_max_depth}")
    best_eta = grid_search.best_params_['eta']
    print(f"Best eta: {best_eta}")

    final_xgb_clf = xgb.XGBClassifier(objective="binary:logistic",
                                      max_depth=best_max_depth,
                                      eta=best_eta,
                                      device=boost_device,
                                      random_state=3137)
    final_xgb_clf.fit(data, labels)

    return final_xgb_clf

In [ ]:
# Train model
xgb_model = train_model(data=train_embeddings_array, labels=y_train)

Number of rows in training data: 20033
Fitting 5 folds for each of 20 candidates, totalling 100 fits


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning:

[01:26:22] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.




[CV 1/5] END .............eta=0.05, max_depth=5;, score=0.889 total time=   4.3s
[CV 2/5] END .............eta=0.05, max_depth=5;, score=0.881 total time=   5.2s
[CV 3/5] END .............eta=0.05, max_depth=5;, score=0.884 total time=   3.6s
[CV 4/5] END .............eta=0.05, max_depth=5;, score=0.879 total time=   4.5s
[CV 5/5] END .............eta=0.05, max_depth=5;, score=0.897 total time=   4.6s
[CV 1/5] END .............eta=0.05, max_depth=6;, score=0.894 total time=   4.9s
[CV 2/5] END .............eta=0.05, max_depth=6;, score=0.886 total time=   4.4s
[CV 3/5] END .............eta=0.05, max_depth=6;, score=0.894 total time=   4.5s
[CV 4/5] END .............eta=0.05, max_depth=6;, score=0.883 total time=   4.6s
[CV 5/5] END .............eta=0.05, max_depth=6;, score=0.905 total time=   4.3s
[CV 1/5] END .............eta=0.05, max_depth=7;, score=0.895 total time=   6.3s
[CV 2/5] END .............eta=0.05, max_depth=7;, score=0.888 total time=   6.9s
[CV 3/5] END .............et

In [ ]:
y_pred = xgb_model.predict(test_embeddings_array)

In [ ]:
y_pred

array([1, 0, 1, ..., 0, 0, 1])

Results
-----------------

In [ ]:
print(f"Classification report:\n{classification_report(y_test, y_pred)}")

Classification report:
              precision    recall  f1-score   support

           0       0.84      0.85      0.84      4496
           1       0.83      0.82      0.82      4090

    accuracy                           0.83      8586
   macro avg       0.83      0.83      0.83      8586
weighted avg       0.83      0.83      0.83      8586



Based on the classification report, the model exhibits high Precision and Recall for both classes. The precision represents the probability that a headline that was predicted as sarcasitic is actually sarcastic and the recall is the probability that an actually sarcastic headline was detected as sarcastic.

In other words, a high precision means that the model creates fewer false positives and higer recall would mean that the model creates fewer false negatives.

In addition, The F1-score, which is just a harmonic mean between the precision and the recall is also high. A high F1-score suggests that the precision and recall are balanced, meaning that false negative and positive errors occur at a similar rate. A lower F1-score means that one type of error occurs more frequently than the other. This could suggest an unbalanced datatset or a faulty model.

Overall, the model detects sarcasm in headlines well.




Conclusion
-------------------------

In conclusion, the XGBoost model trained on RoBERTa  embeddings successfully classifies headlines as sarcastic or not.

Due to it's high performance, this model can be used in social media content moderation and sentiment analysis where sarcasm detection is required.

For content moderation, this model can be used to aid in tasks such as detecting hate speech or flagging inapproriate content. In addition to sarcasm detection, models built with text embeddings can be used to determine other emotional tones of text. These can be used to predict overall sentiments in online conversations such as positive, negative and neutral.

However, this model can perform better with the following potential improvements.
Experimenting with other embedding models such as BERT, XLNet or ELMo could yield better scores in the classification report. Furthermore, expanding the dataset could expose the model to more sarcastic expressions and could expand the scope of it's detecting abilities.

Bibliography

Chen, Tianqi & Guestrin, Carlos. (2016). XGBoost: A Scalable Tree Boosting System. 785-794. 10.1145/2939672.2939785.

Sneha, Pianalytix-. « XGBoost | Machine Learning ». Pianalytix - Build Real-World Tech Projects, 17 novembre 2020, https://pianalytix.com/xgboost-machine-learning/

Starmer, Josh-. XGBoost Part 2 (of 4): Classification.YouTube.https://www.youtube.com/watch?si=OF1oOJ9iyr7FG_di&v=8b1JEDvenQU&feature=youtu.be. Consulté le 23 novembre 2024.

Introduction to Boosted Trees — xgboost 2.1.1 documentation. https://xgboost.readthedocs.io/en/stable/tutorials/model.html. Consulté le 3 décembre 2024.

Ghosh, Suvrodeep. « Leveraging Text Embeddings for Twitter Text Classification with XGBoost ». Medium, 1 avril 2024, https://medium.com/@suvro.dgp/leveraging-text-embeddings-for-twitter-text-classification-with-xgboost-6a4a1a89371c.

Flair | Flair. https://flairnlp.github.io/. Consulté le 9 décembre 2024.